In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from Mask_RCNN.mrcnn import utils
import Mask_RCNN.mrcnn.model as modellib
from Mask_RCNN.mrcnn import visualize
# Import COCO config
from Mask_RCNN.samples.coco import coco

%matplotlib inline 

ModuleNotFoundError: No module named 'Mask_RCNN'

In [9]:



class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

ModuleNotFoundError: No module named 'coco'

In [10]:
# output = "final_output.mp4"
clip1 = VideoFileClip("../original_clip.mp4")
# clip1.write_videofile(output, audio=False)